In [2]:
import pandas as pd

# Data cleaning process for weather-related data

In [16]:
weather_raw = pd.read_csv("load_data/weather_raw.csv", low_memory=False)
weather_raw["DATE"] = pd.to_datetime(weather_raw["DATE"])

In [18]:
weather_raw.head()

,STATION,DATE,REPORT_TYPE,TMP,WND
0,71877099999,2011-01-01 00:00:00,SY-MT,"-0094,1","320,1,N,0036,1"
1,71393099999,2011-01-01 00:00:00,FM-12,"-0098,1","999,9,9,9999,9"
2,71235099999,2011-01-01 00:00:00,SY-SA,"-0114,1","320,1,N,0046,1"
3,71877099999,2011-01-01 01:00:00,FM-15,"-0090,1","320,1,N,0041,1"
4,71235099999,2011-01-01 01:00:00,SAO,"-0120,1","320,1,N,0041,1"


Note: The timestamps in raw data are in UTC time zone. 

In [10]:
# Round times (convert all timestamps to on-the-hour for better merging in the following steps)
weather_raw["DATE"] = weather_raw["DATE"].dt.round("H")
# Convert from UTC to Mountain time
weather_raw["DATE"] = pd.to_datetime(weather_raw["DATE"]).dt.tz_localize("UTC").dt.tz_convert("America/Denver").dt.tz_localize(None)

### Parse TMP and WND into numerical values in degree Celcius and meter/sec, according to ISD format document.

In [20]:
# Parse temperature records into Celcius
def parse_temp(temp_str):
    temp_raw = temp_str.split(',')[0]
    sign = -1 if temp_raw[0] == '-' else 1
    value = int(temp_raw[1:4]) + int(temp_raw[4])/10
    return sign*value
weather_raw['Temp_C'] = weather_raw['TMP'].apply(parse_temp)

In [22]:
# Parse wind speed records into mps
def parse_wnd(wnd_str):
    parts = wnd_str.split(',')
    if len(parts) != 5:
        return 999  # invalid format
    
    direction, quality_dir, type_code, speed, quality_speed = parts
    
    try:
        speed = int(speed)  
    except ValueError:
        return 999  # invalid data

    # quality code check: 2, 3, 6, 7 indicates erroneous value
    if quality_dir in {'2', '3', '6', '7'} or quality_speed in {'2', '3', '6', '7'}:
        return 999
    # speed check: 9999 indicates missing data
    if speed == 9999:
        return 999
    # type check: 9 indicates missing data
    if type_code == '9':
        return 999

    # parse speed：convert to mps
    return speed / 10

weather_raw['Wind_mps'] = weather_raw['WND'].apply(parse_wnd)

In [24]:
# Flag erroneous temperature

# parse 'Month' into a column
weather_raw["Month"] = pd.to_datetime(weather_raw["DATE"]).dt.month

# check if temperature is valid 
def check_temp_valid(row):
    temp = row["Temp_C"]
    month = row["Month"]

    # (high temperature in winter or abnormally high temp (>38) in general indicates invalid data. 
    if (month in [12, 1, 2] and temp > 15.0) or \
       (month in [11, 3] and temp > 25.0) or \
       (temp > 38.0):
        return False
    return True

# Write the flags into a separate column
weather_raw["Temp_valid"] = weather_raw.apply(check_temp_valid, axis=1)

# Delete temporary column 'Month'
weather_raw.drop(columns=["Month"], inplace=True)

# Flag erroneous wind speed
def check_wind_valid(row):
    wind = row['Wind_mps']
    if wind == 999:
        return False
    return True
    
weather_raw["Wind_valid"] = weather_raw.apply(check_wind_valid, axis=1)

In [26]:
weather_raw.head()

,STATION,DATE,REPORT_TYPE,TMP,WND,Temp_C,Wind_mps,Temp_valid,Wind_valid
0,71877099999,2011-01-01 00:00:00,SY-MT,"-0094,1","320,1,N,0036,1",-9.4,3.6,True,True
1,71393099999,2011-01-01 00:00:00,FM-12,"-0098,1","999,9,9,9999,9",-9.8,999.0,True,False
2,71235099999,2011-01-01 00:00:00,SY-SA,"-0114,1","320,1,N,0046,1",-11.4,4.6,True,True
3,71877099999,2011-01-01 01:00:00,FM-15,"-0090,1","320,1,N,0041,1",-9.0,4.1,True,True
4,71235099999,2011-01-01 01:00:00,SAO,"-0120,1","320,1,N,0041,1",-12.0,4.1,True,True


In [28]:
# filter valid temperature and wind speed data
valid_temp = weather_raw[weather_raw["Temp_valid"]][["DATE", "Temp_C"]]
valid_wind = weather_raw[weather_raw["Wind_valid"]][["DATE", "Wind_mps"]]

# aggregate by time (if more than one valid records for an hour, use the average)
temp_avg = valid_temp.groupby("DATE", as_index=False).agg({"Temp_C": "mean"})
wind_avg = valid_wind.groupby("DATE", as_index=False).agg({"Wind_mps": "mean"})

# merge data to keep only valid points
weather_aggregated = pd.merge(temp_avg, wind_avg, on="DATE", how="outer")

In [30]:
# Keep only one decimal place
weather_aggregated["Temp_C"] = weather_aggregated["Temp_C"].round(1)
weather_aggregated["Wind_mps"] = weather_aggregated["Wind_mps"].round(1)

In [32]:
weather_aggregated.head()

,DATE,Temp_C,Wind_mps
0,2011-01-01 00:00:00,-10.2,4.1
1,2011-01-01 01:00:00,-10.5,4.1
2,2011-01-01 02:00:00,-11.5,3.4
3,2011-01-01 03:00:00,-11.0,2.8
4,2011-01-01 04:00:00,-10.0,1.5


### Check for and handle missing data
##### For short missing periods (<24 hr), fill by linear interpolation
##### For long missing periods (>= 24 hr), find secondary data source 

In [34]:
# Check for missing times and duplicates
# Full time range 2011-01-01 0:00 - 2024-12-31 23:00
full_range = pd.date_range(start=weather_aggregated['DATE'].min(), end=weather_aggregated['DATE'].max(), freq='H')

# Find missing points
missing_times = full_range.difference(weather_aggregated['DATE'])

# Find duplicated points
duplicated_times = weather_aggregated['DATE'][weather_aggregated['DATE'].duplicated(keep=False)]

print("Missing times:")
print(missing_times)

print("\nDuplicated times:")
print(duplicated_times.unique())

Missing times:
DatetimeIndex(['2011-05-04 11:00:00', '2011-05-31 14:00:00',
               '2011-05-31 16:00:00', '2011-08-12 16:00:00',
               '2011-10-07 13:00:00', '2011-11-25 02:00:00',
               '2012-03-20 20:00:00', '2012-03-20 21:00:00',
               '2012-03-20 22:00:00', '2012-03-20 23:00:00',
               ...
               '2024-12-03 18:00:00', '2024-12-03 19:00:00',
               '2024-12-03 20:00:00', '2024-12-03 21:00:00',
               '2024-12-03 23:00:00', '2024-12-18 09:00:00',
               '2024-12-18 10:00:00', '2024-12-18 11:00:00',
               '2024-12-18 12:00:00', '2024-12-18 13:00:00'],
              dtype='datetime64[ns]', length=867, freq=None)

Duplicated times:
[]


In [36]:
# Check for one missing data (temp or wind)
missing_wind = weather_aggregated[weather_aggregated['Wind_mps'].isna()]
missing_temp = weather_aggregated[weather_aggregated['Temp_C'].isna()]

print("Missing Wind:")
print(missing_wind)
print("\nMissing Temperature:")
print(missing_temp)

Missing Wind:
                      DATE  Temp_C  Wind_mps
22813  2013-04-17 08:00:00    -7.0       NaN
23571  2013-05-15 23:00:00    17.4       NaN
23930  2013-05-28 17:00:00    15.0       NaN
24339  2013-06-09 19:48:00     9.0       NaN
24450  2013-06-13 20:00:00    10.3       NaN
...                    ...     ...       ...
139594 2024-09-28 08:00:00     8.2       NaN
139608 2024-09-28 23:00:00    16.3       NaN
141122 2024-11-26 21:00:00    -2.8       NaN
141972 2024-12-25 04:00:00     2.6       NaN
142058 2024-12-28 08:00:00    -4.6       NaN

[7497 rows x 3 columns]

Missing Temperature:
                      DATE  Temp_C  Wind_mps
142177 2011-03-29 11:29:00     NaN       3.1
142178 2012-05-03 13:41:00     NaN       4.1
142179 2012-05-03 14:32:00     NaN       5.1
142180 2012-05-03 16:30:00     NaN       4.1
142181 2012-05-03 17:27:00     NaN       5.1
142182 2012-05-03 22:30:00     NaN       7.7
142183 2012-05-03 23:30:00     NaN       9.3
142184 2012-05-04 22:30:00     NaN     

In [38]:
# Merge missing timestamps
weather_aggregated['DATE'] = pd.to_datetime(weather_aggregated['DATE'])
weather_complete = pd.DataFrame({'DATE': full_range}).merge(weather_aggregated, on='DATE', how='left')
weather_complete.to_csv("load_data/weather_processed2_merged.csv")
weather_complete.head()

,DATE,Temp_C,Wind_mps
0,2011-01-01 00:00:00,-10.2,4.1
1,2011-01-01 01:00:00,-10.5,4.1
2,2011-01-01 02:00:00,-11.5,3.4
3,2011-01-01 03:00:00,-11.0,2.8
4,2011-01-01 04:00:00,-10.0,1.5


In [40]:
weather_complete['DATE'] = pd.to_datetime(weather_complete['DATE'])
weather_complete = weather_complete.sort_values(by='DATE')

In [44]:
def find_long_missing_periods(df, column, min_gap=24):
    """
    
    Find in column for long missing periods (over min_gap hours) 
    """
    missing = df[column].isna()  # find missing values
    missing_groups = (missing != missing.shift()).cumsum()  
    gap_sizes = missing.groupby(missing_groups).transform('sum')  # compute the length of missing period
    long_missing = (missing & (gap_sizes >= min_gap))  # keep the time periods of over min_gap hours
    
    return df.loc[long_missing, ['DATE', column]]  

# Find missing periods of over 24 hours for temperature data
long_missing_temp = find_long_missing_periods(weather_complete, 'Temp_C', min_gap=24)

# Find missing periods of over 24 hours for wind speed data
long_missing_wind = find_long_missing_periods(weather_complete, 'Wind_mps', min_gap=24)

long_missing_temp.to_csv("missing_temp.csv", index=False)
long_missing_wind.to_csv("missing_wind.csv", index=False)

df = pd.read_csv("missing_temp.csv")
df.head()

,DATE,Temp_C
0,2018-02-20 16:00:00,NaN
1,2018-02-20 17:00:00,NaN
2,2018-02-20 18:00:00,NaN
3,2018-02-20 19:00:00,NaN
4,2018-02-20 20:00:00,NaN


In [46]:
# Find missing periods of over 24 hours for temperature data

df['DATE'] = pd.to_datetime(df['DATE'])
df = df.sort_values('DATE').reset_index(drop=True)

# compute length of missing period (in hour)
df['Time_Diff'] = df['DATE'].diff().dt.total_seconds() / 3600

group_id = 0
group_ids = []
for diff in df['Time_Diff'].fillna(float('inf')):
    if diff == 1:
        group_ids.append(group_id)
    else:
        group_id += 1
        group_ids.append(group_id)
df['Group_ID'] = group_ids

# Filter for missing periods of over 24 hours
group_sizes = df.groupby('Group_ID').size()
valid_groups = group_sizes[group_sizes > 24].index

# Extract start/end time of each period
result = df[df['Group_ID'].isin(valid_groups)]
grouped = result.groupby('Group_ID')['DATE'].agg(['min', 'max', 'count'])

print("Long missing period of temperature")
print(grouped.rename(columns={'min': 'Start', 'max': 'End', 'count': 'Length'}))

Long missing period of temperature
                       Start                 End  Length
Group_ID                                                
1        2018-02-20 16:00:00 2018-02-21 17:00:00      26
2        2021-02-12 07:00:00 2021-02-13 09:00:00      27
3        2021-02-19 12:00:00 2021-02-20 14:00:00      27
4        2024-03-27 18:00:00 2024-04-01 05:00:00     108
5        2024-11-17 22:00:00 2024-11-21 00:00:00      75


In [48]:
# Find missing periods of over 24 hours for wind speed data

df_wind = pd.read_csv("missing_wind.csv")
df_wind['DATE'] = pd.to_datetime(df_wind['DATE'])

# 排序时间
df_wind = df_wind.sort_values('DATE').reset_index(drop=True)

# 计算时间差（小时）
df_wind['Time_Diff'] = df_wind['DATE'].diff().dt.total_seconds() / 3600

# 初始化分组ID
group_id_wind = 0
group_ids_wind = []
for diff in df_wind['Time_Diff'].fillna(float('inf')):
    if diff == 1:
        group_ids_wind.append(group_id_wind)
    else:
        group_id_wind += 1
        group_ids_wind.append(group_id_wind)
df_wind['Group_ID'] = group_ids_wind

# 统计每组的长度，筛选长度大于10的
group_sizes = df_wind.groupby('Group_ID').size()
valid_groups = group_sizes[group_sizes > 24].index

# 提取每个有效连续段的起始和结束时间
result = df_wind[df_wind['Group_ID'].isin(valid_groups)]
grouped = result.groupby('Group_ID')['DATE'].agg(['min', 'max', 'count'])

# 打印每段的起止时间和长度
print("Long missing period of wind")
print(grouped.rename(columns={'min': 'Start', 'max': 'End', 'count': 'Length'}))

Long missing period of wind
                       Start                 End  Length
Group_ID                                                
1        2018-02-20 16:00:00 2018-02-21 17:00:00      26
2        2021-02-12 07:00:00 2021-02-13 09:00:00      27
3        2021-02-19 12:00:00 2021-02-20 14:00:00      27
4        2024-03-27 18:00:00 2024-04-01 05:00:00     108
5        2024-11-17 22:00:00 2024-11-21 00:00:00      75


In [50]:
def mask_long_gaps(series, max_gap=24):
    """Flag short missing periods (< max_gap hours)"""
    is_na = series.isna()
    group = (~is_na).cumsum() 
    gap_sizes = is_na.groupby(group).transform('sum')  
    return (gap_sizes < max_gap)  

# Flag short missing periods for interpolation
mask_temp = mask_long_gaps(weather_complete['Temp_C'], max_gap=24)
mask_wind = mask_long_gaps(weather_complete['Wind_mps'], max_gap=24)

# Linear interpolation for short missing periods
weather_complete.loc[mask_temp, 'Temp_C'] = weather_complete['Temp_C'].interpolate(method='linear')
weather_complete.loc[mask_wind, 'Wind_mps'] = weather_complete['Wind_mps'].interpolate(method='linear')

Long missing periods are filled with data from a secondary online climate database, Weather Spark (https://weatherspark.com).
Now the weather-related data is complete.

In [63]:
weather_complete.to_csv("load_data/weather_complete_.csv", index=False)

# Data cleaning process for load data

### Load 1 - 2011-01-01 - 2016-12-31

In [68]:
load1 = pd.read_csv("load_data/load1.csv", low_memory=False) # Load data 2011-01-01 - 2016-12-31
load1.head()

,DATE,HOUR ENDING,CALGARY,HOUR
0,2011-01-01,1.0,1045.0,12:00:00 AM
1,2011-01-01,2.0,996.0,1:00:00 AM
2,2011-01-01,3.0,957.0,2:00:00 AM
3,2011-01-01,4.0,935.0,3:00:00 AM
4,2011-01-01,5.0,927.0,4:00:00 AM


In [70]:
load1["Datetime"] = pd.to_datetime(load1["DATE"] + " " + load1["HOUR"])
load1 = load1.drop(columns=["DATE", "HOUR", "HOUR ENDING"])
load1.dropna(how='all')
load1.head()

,CALGARY,Datetime
0,1045.0,2011-01-01 00:00:00
1,996.0,2011-01-01 01:00:00
2,957.0,2011-01-01 02:00:00
3,935.0,2011-01-01 03:00:00
4,927.0,2011-01-01 04:00:00


### Check for and handle missing data

In [73]:
# Check for missing times and duplicates
# Full time range 2011-01-01 0:00 - 2016-12-31 23:00
full_range = pd.date_range(start=load1["Datetime"].min(), end=load1["Datetime"].max(), freq='H')

# Find missing points
missing_times = full_range.difference(load1["Datetime"])

# Find duplicated points
duplicated_times = load1['Datetime'][load1['Datetime'].duplicated(keep=False)]

print("Missing times:")
print(missing_times)

print("\nDuplicated times:")
print(duplicated_times.unique())

Missing times:
DatetimeIndex(['2011-03-13 01:00:00', '2012-03-11 01:00:00',
               '2013-03-10 01:00:00', '2014-03-09 01:00:00',
               '2015-03-08 01:00:00', '2016-03-13 01:00:00'],
              dtype='datetime64[ns]', freq=None)

Duplicated times:
['NaT']


No continuous missing time periods. All gaps can be filled by linear interpolation

In [81]:
# Merge missing times
load1["Datetime"] = pd.to_datetime(load1["Datetime"])
load1_complete = pd.DataFrame({'Datetime': full_range}).merge(load1, on='Datetime', how='left')

#Fill missing data
load1_complete = load1_complete.sort_values("Datetime")
load1_complete["CALGARY"] = load1_complete["CALGARY"].interpolate(method='linear')

load1_complete.to_csv("load_data/load1_complete.csv", index=False)

## Load 2 - 2017-01-01 - 2020-04-30

In [83]:
load2 = pd.read_csv("load_data/load2.csv") 
load2.head()

,DATE,HOUR ENDING,CALGARY
0,2017-01-01,1,1062
1,2017-01-01,2,1017
2,2017-01-01,3,983
3,2017-01-01,4,961
4,2017-01-01,5,951


In [89]:
load2["Hour"] = load2["HOUR ENDING"].astype(int) - 1
load2["Datetime"] = pd.to_datetime(load2["DATE"]) + pd.to_timedelta(load2["Hour"], unit="h")
load2.head()

,DATE,HOUR ENDING,CALGARY,Hour,Datetime
0,2017-01-01,1,1062,0,2017-01-01 00:00:00
1,2017-01-01,2,1017,1,2017-01-01 01:00:00
2,2017-01-01,3,983,2,2017-01-01 02:00:00
3,2017-01-01,4,961,3,2017-01-01 03:00:00
4,2017-01-01,5,951,4,2017-01-01 04:00:00


In [91]:
load2 = load2.drop(columns=["DATE", "Hour", "HOUR ENDING"])

### Check for and handle missing data

In [97]:
# Check for missing times and duplicates
# Full time range 2011-01-01 0:00 - 2024-12-31 23:00
full_range = pd.date_range(start=load2["Datetime"].min(), end=load2["Datetime"].max(), freq='H')

# Find missing points
missing_times = full_range.difference(load2["Datetime"])

# Find duplicated points
duplicated_times = load2['Datetime'][load2['Datetime'].duplicated(keep=False)]

print("Missing times:")
print(missing_times)

print("\nDuplicated times:")
print(duplicated_times.unique())

Missing times:
DatetimeIndex(['2017-03-12 01:00:00', '2018-03-11 01:00:00',
               '2019-03-10 01:00:00', '2020-03-08 01:00:00'],
              dtype='datetime64[ns]', freq=None)

Duplicated times:
[]


In [99]:
# Merge missing times
load2["Datetime"] = pd.to_datetime(load2["Datetime"])
load2_complete = pd.DataFrame({'Datetime': full_range}).merge(load2, on='Datetime', how='left')

#Fill missing data
load2_complete = load2_complete.sort_values("Datetime")
load2_complete["CALGARY"] = load2_complete["CALGARY"].interpolate(method='linear')

load2_complete.to_csv("load_data/load2_complete.csv", index=False)

## Load 3 - 2020-05-01 - 2022-12-31

In [103]:
load3 = pd.read_csv("load_data/load3.csv")
load3.head()

,Datetime,Calgary
0,2020-05-01 00:00:00,829.528569
1,2020-05-01 01:00:00,811.903073
2,2020-05-01 02:00:00,806.679480
3,2020-05-01 03:00:00,816.909192
4,2020-05-01 04:00:00,858.466233


### Check for and handle missing data

In [106]:
# Check for missing times and duplicates
# Full time range 2011-01-01 0:00 - 2024-12-31 23:00
full_range = pd.date_range(start=load3["Datetime"].min(), end=load3["Datetime"].max(), freq='H')

# Find missing points
missing_times = full_range.difference(load3["Datetime"])

# Find duplicated points
duplicated_times = load3['Datetime'][load3['Datetime'].duplicated(keep=False)]

print("Missing times:")
print(missing_times)

print("\nDuplicated times:")
print(duplicated_times.unique())

Missing times:
DatetimeIndex([], dtype='datetime64[ns]', freq=None)

Duplicated times:
[]


In [108]:
load3["Datetime"] = pd.to_datetime(load3["Datetime"])
load3.to_csv("load_data/load3_complete.csv", index=False)

## Load 4 - 2023-01-01 - 2024-12-31

In [112]:
load4 = pd.read_csv("load_data/load4.csv")
load4.head()

,Datetime,Calgary
0,2023-11-01 0:00,939.010971
1,2023-11-01 1:00,916.904998
2,2023-11-01 2:00,921.081268
3,2023-11-01 3:00,944.987700
4,2023-11-01 4:00,1011.696600


In [114]:
# Check for missing times and duplicates
# Full time range 2011-01-01 0:00 - 2024-12-31 23:00
full_range = pd.date_range(start=load4["Datetime"].min(), end=load4["Datetime"].max(), freq='H')

# Find missing points
missing_times = full_range.difference(load4["Datetime"])

# Find duplicated points
duplicated_times = load4['Datetime'][load4['Datetime'].duplicated(keep=False)]

print("Missing times:")
print(missing_times)

print("\nDuplicated times:")
print(duplicated_times.unique())

Missing times:
DatetimeIndex([], dtype='datetime64[ns]', freq=None)

Duplicated times:
[]


In [116]:
load4["Datetime"] = pd.to_datetime(load4["Datetime"])
load4.to_csv("load_data/load4_complete.csv", index=False)

# Combine load and weather data

In [192]:
weather = pd.read_csv("load_data/weather_final.csv", low_memory=False)
weather.head()

,Datetime,Temp_C,Wind_mps
0,2011-01-01 0:00,-10.0,2.6
1,2011-01-01 1:00,-9.5,2.6
2,2011-01-01 2:00,-11.0,3.1
3,2011-01-01 3:00,-13.0,2.0
4,2011-01-01 4:00,-15.0,2.0


In [194]:
load1 = pd.read_csv("load_data/load1_complete.csv", low_memory=False)
load2 = pd.read_csv("load_data/load2_complete.csv", low_memory=False)
load3 = pd.read_csv("load_data/load3_complete.csv", low_memory=False)
load4 = pd.read_csv("load_data/load4_complete.csv", low_memory=False)

In [196]:
for df in [load1, load2, load3, load4, weather]:
    df["Datetime"] = pd.to_datetime(df["Datetime"])

# Concatenate all load data
load_all = pd.concat([load1, load2, load3, load4], axis=0)
load_all = load_all.sort_values("Datetime")

# Merge load and weather data by date time
data = pd.merge(load_all, weather, on="Datetime", how="left")
data.head()

,Datetime,Load,Temp_C,Wind_mps
0,2011-01-01 00:00:00,1045.0,-10.0,2.6
1,2011-01-01 01:00:00,996.0,-9.5,2.6
2,2011-01-01 02:00:00,957.0,-11.0,3.1
3,2011-01-01 03:00:00,935.0,-13.0,2.0
4,2011-01-01 04:00:00,927.0,-15.0,2.0


In [202]:
# Check for missing times and duplicates
# Full time range 2011-01-01 0:00 - 2024-12-31 23:00
full_range = pd.date_range(start=data["Datetime"].min(), end=data["Datetime"].max(), freq='H')

# Find missing points
missing_times = full_range.difference(data["Datetime"])

# Find duplicated points
duplicated_times = data['Datetime'][data['Datetime'].duplicated(keep=False)]

print("Missing times:")
print(missing_times)

print("\nDuplicated times:")
print(duplicated_times.unique())

Missing times:
DatetimeIndex([], dtype='datetime64[ns]', freq=None)

Duplicated times:
[]


# Time-related data columns

In [233]:
data["Datetime"] = pd.to_datetime(data["Datetime"])

data["Day_of_Week_Num"] = data["Datetime"].dt.weekday # day indicator 0-6 Monday-Sunday

data["Is_Weekend"] = data["Day_of_Week_Num"].apply(lambda x: 1 if x >= 5 else 0) # Binary weekend indicator

data.head()

,Datetime,Load_Calgary,Temperature_C,Wind_Speed_mps,Day_of_Week_Num,Is_Weekend
0,2011-01-01 00:00:00,1045.0,-10.0,2.6,5,1
1,2011-01-01 01:00:00,996.0,-9.5,2.6,5,1
2,2011-01-01 02:00:00,957.0,-11.0,3.1,5,1
3,2011-01-01 03:00:00,935.0,-13.0,2.0,5,1
4,2011-01-01 04:00:00,927.0,-15.0,2.0,5,1


In [231]:
data.drop(columns=['Is_Weekend', 'Day_of_Week'], inplace=True)

In [235]:
# All statutory holidays in Alberta between 2011-2024
holiday_list = [
    "2011-01-01", "2011-02-21", "2011-04-22",  
    "2011-05-23", "2011-07-01", "2011-09-05",
    "2011-10-10", "2011-11-11", "2011-12-25", 
    "2012-01-01", "2012-02-20", "2012-04-06",  
    "2012-05-21", "2012-07-01", "2012-09-03",
    "2012-10-08", "2012-11-11", "2012-12-25", 
    "2013-01-01", "2013-02-18", "2013-03-29",  
    "2013-05-20", "2013-07-01", "2013-09-02",
    "2013-10-14", "2013-11-11", "2013-12-25", 
    "2014-01-01", "2014-02-17", "2014-04-18",  
    "2014-05-19", "2014-07-01", "2014-09-01",
    "2014-10-13", "2014-11-11", "2014-12-25", 
    "2015-01-01", "2015-02-16", "2015-04-03",  
    "2015-05-18", "2015-07-01", "2015-09-07",
    "2015-10-12", "2015-11-11", "2015-12-25",
    "2016-01-01", "2016-02-15", "2016-03-25",  
    "2016-05-23", "2016-07-01", "2016-09-05",
    "2016-10-10", "2016-11-11", "2016-12-25",
    "2017-01-01", "2017-02-20", "2017-04-14",  
    "2017-05-22", "2017-07-01", "2017-09-04",
    "2017-10-09", "2017-11-11", "2017-12-25",
    "2018-01-01", "2018-02-19", "2018-03-30",  
    "2018-05-21", "2018-07-01", "2018-09-03",
    "2018-10-08", "2018-11-11", "2018-12-25",
    "2019-01-01", "2019-02-18", "2019-04-19",  
    "2019-05-20", "2019-07-01", "2019-09-02",
    "2019-10-14", "2019-11-11", "2019-12-25",
    "2020-01-01", "2020-02-17", "2020-04-10",  
    "2020-05-18", "2020-07-01", "2020-09-07",
    "2020-10-12", "2020-11-11", "2020-12-25",
    "2021-01-01", "2021-02-15", "2021-04-02",  
    "2021-05-24", "2021-07-01", "2021-09-06",
    "2021-10-11", "2021-11-11", "2021-12-25",
    "2022-01-01", "2022-02-21", "2022-04-15",  
    "2022-05-23", "2022-07-01", "2022-09-05",
    "2022-10-10", "2022-11-11", "2022-12-25",
    "2023-01-01", "2023-02-20", "2023-04-07",  
    "2023-05-22", "2023-07-01", "2023-09-04",
    "2023-10-09", "2023-11-11", "2023-12-25",
    "2024-01-01", "2024-02-19", "2024-03-29",  
    "2024-05-20", "2024-07-01", "2024-09-02",
    "2024-10-14", "2024-11-11", "2024-12-25",
]

holiday_dates = pd.to_datetime(holiday_list).date
data["Date_only"] = data["Datetime"].dt.date

# Binary holiday indicator
data["Is_Holiday"] = data["Date_only"].apply(lambda x: 1 if x in holiday_dates else 0)

data.drop(columns=["Date_only"], inplace=True)

data.head()

,Datetime,Load_Calgary,Temperature_C,Wind_Speed_mps,Day_of_Week_Num,Is_Weekend,Is_Holiday
0,2011-01-01 00:00:00,1045.0,-10.0,2.6,5,1,1
1,2011-01-01 01:00:00,996.0,-9.5,2.6,5,1,1
2,2011-01-01 02:00:00,957.0,-11.0,3.1,5,1,1
3,2011-01-01 03:00:00,935.0,-13.0,2.0,5,1,1
4,2011-01-01 04:00:00,927.0,-15.0,2.0,5,1,1


In [237]:
import numpy as np

# Convert day of week into sinusoidal values to include circular behaviour
data["Day_of_Week_Sin"] = np.sin(2 * np.pi * data["Day_of_Week_Num"] / 7)
data.head()

,Datetime,Load_Calgary,Temperature_C,Wind_Speed_mps,Day_of_Week_Num,Is_Weekend,Is_Holiday,Day_of_Week_Sin
0,2011-01-01 00:00:00,1045.0,-10.0,2.6,5,1,1,-0.974928
1,2011-01-01 01:00:00,996.0,-9.5,2.6,5,1,1,-0.974928
2,2011-01-01 02:00:00,957.0,-11.0,3.1,5,1,1,-0.974928
3,2011-01-01 03:00:00,935.0,-13.0,2.0,5,1,1,-0.974928
4,2011-01-01 04:00:00,927.0,-15.0,2.0,5,1,1,-0.974928


In [239]:
data.to_csv("load_data/data_complete.csv", index=False)